### kNNs

This will be the fallback approach. If a new user who has never been in the database comes along with a list of anime they liked, how are we going to use the SVD algo? We can't unless we put this new data point into the dataset and retrain. But that is slow. Therefore, we will need to have a different, fallback method. To keep things simple, we can use only the genre type as a feature of the anime and try to compute distance metrics on those features between different anime. Close anime are similar anime we will recommend. 

In [1]:
import pandas as pd
import pickle

/tmp/ipykernel_202690/3842437144.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
userID2userNameMap = {}
userName2userIDMap = {}

animeID2animeNameMap = {}
animeName2animeIDMap = {}

with open('userName2userIDMap.pkl', 'rb') as f:
    userName2userIDMap = pickle.load(f)

with open('userID2userNameMap.pkl', 'rb') as f:
    userID2userNameMap = pickle.load(f)


with open('animeID2animeNameMap.pkl', 'rb') as f:
    animeID2animeNameMap = pickle.load(f)

with open('animeName2animeIDMap.pkl', 'rb') as f:
    animeName2animeIDMap = pickle.load(f)

In [3]:
df = None
with open('df_custom.pkl', 'rb') as f:
    df = pickle.load(f)

In [4]:
df

,user_id,anime_id,score
0,2255153,21,9
1,2255153,59,7
2,2255153,74,7
3,2255153,120,7
4,2255153,178,7
...,...,...,...
31284025,4862000,15611,9
31284026,4862000,27815,9
31284027,299167,5945,8
31284028,263803,1316,9


In [5]:
userID2userNameMap[1]

'Xinil'